In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/padhai-mp-neuron-like-unlike-classification/sample_submission.csv
/kaggle/input/padhai-mp-neuron-like-unlike-classification/train.csv
/kaggle/input/padhai-mp-neuron-like-unlike-classification/test.csv


In [2]:
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,StandardScaler
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error,log_loss
import operator
import json
from IPython import display 
import os
import warnings
np.random.seed(0)
warnings.filterwarnings("ignore")
THRESHOLD = 4

In [3]:
train = pd.read_csv("/kaggle/input/padhai-mp-neuron-like-unlike-classification/train.csv")
test = pd.read_csv("/kaggle/input/padhai-mp-neuron-like-unlike-classification/test.csv")

In [4]:
train.shape

(355, 99)

In [5]:
train.head(5)

,PhoneId,Also Known As,Applications,Architecture,Aspect Ratio,Audio Features,Audio Jack,Autofocus,Bezel-less display,Bluetooth,...,Video Recording,Video Recording Features,VoLTE,Waterproof,Weight,Wi-Fi,Wi-Fi Features,Width,Wireless Charging,Rating
0,0,NaN,NaN,64 bit,19:9,NaN,3.5 mm,Phase Detection autofocus,yes,v5.0,...,1920x1080 @ 30 fps,NaN,yes,NaN,182 grams Below Average ▾Weight compared to11 ...,"Wi-Fi 802.11, a/b/g/n","Wi-Fi Direct, Mobile Hotspot",76.4 mm,NaN,4.5
1,1,NaN,Oppo Browser,64 bit,19:9,NaN,3.5 mm,Phase Detection autofocus,yes,v4.2,...,1920x1080 @ 30 fps,NaN,yes,NaN,168 grams Average ▾Weight compared to8 - 12 K ...,"Wi-Fi 802.11, b/g/n",Mobile Hotspot,75.6 mm,NaN,4.5
2,2,NaN,NaN,64 bit,19.5:9,NaN,3.5 mm,yes,yes,v4.2,...,NaN,NaN,yes,NaN,168 grams Average ▾Weight compared to10 - 14 K...,"Wi-Fi 802.11, b/g/n",Mobile Hotspot,74 mm,NaN,4.4
3,4,NaN,NaN,64 bit,18.5:9,Dolby Atmos,3.5 mm,No,yes,v5.0,...,1920x1080 @ 30 fps,NaN,yes,NaN,169 grams Average ▾Weight compared to15 - 23 K...,"Wi-Fi 802.11, a/ac/b/g/n","Wi-Fi Direct, Mobile Hotspot",76.8 mm,NaN,4.3
4,5,NaN,NaN,64 bit,19.5:9,NaN,3.5 mm,Phase Detection autofocus,yes,v4.2,...,1920x1080 @ 60 fps,NaN,yes,NaN,175 grams Below Average ▾Weight compared to12 ...,"Wi-Fi 802.11, a/ac/b/g/n",Mobile Hotspot,76.6 mm,NaN,4.4


In [6]:
train.columns

Index(['PhoneId', 'Also Known As', 'Applications', 'Architecture',
       'Aspect Ratio', 'Audio Features', 'Audio Jack', 'Autofocus',
       'Bezel-less display', 'Bluetooth', 'Brand', 'Browser', 'Build Material',
       'Camera Features', 'Capacity', 'Chipset', 'Co-Processor', 'Colours',
       'Custom UI', 'Display Colour', 'Display Type', 'Email',
       'Expandable Memory', 'FM Radio', 'Fingerprint Sensor',
       'Fingerprint Sensor Position', 'Flash', 'GPS', 'Games', 'Graphics',
       'HDMI', 'Heart Rate Monitor', 'Height', 'IRIS Scanner',
       'Image Resolution', 'Internal Memory', 'Java', 'Launch Date',
       'Loudspeaker', 'Mobile High-Definition Link(MHL)', 'Model', 'Music',
       'NFC', 'Network', 'Network Support', 'Operating System',
       'Optical Image Stabilisation', 'Other Facilities', 'Other Sensors',
       'Phone Book', 'Physical Aperture', 'Pixel Density', 'Processor',
       'Quick Charging', 'RAM', 'Rating Count', 'Resolution', 'Review Count',
       'Ring

In [7]:
def data_clean(data):
    
    columns_to_remove = ['Also Known As','Applications','Audio Features','Bezel-less display'
                         'Browser','Build Material','Co-Processor','Browser'
                         'Display Colour','Mobile High-Definition Link(MHL)',
                         'Music', 'Email','Fingerprint Sensor Position',
                         'Games','HDMI','Heart Rate Monitor','IRIS Scanner', 
                         'Optical Image Stabilisation','Other Facilities',
                         'Phone Book','Physical Aperture','Quick Charging',
                         'Ring Tone','Ruggedness','SAR Value','SIM 3','SMS',
                         'Screen Protection','Screen to Body Ratio (claimed by the brand)',
                         'Sensor','Software Based Aperture', 'Special Features',
                         'Standby time','Stylus','TalkTime', 'USB Type-C',
                         'Video Player', 'Video Recording Features','Waterproof',
                         'Wireless Charging','USB OTG Support', 'Video Recording','Java']

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    #Features having very low variance 
    columns_to_remove = ['Architecture','Audio Jack','GPS','Loudspeaker','Network','Network Support','VoLTE']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    # Multivalued:
    columns_to_remove = ['Architecture','Launch Date','Audio Jack','GPS','Loudspeaker','Network','Network Support','VoLTE', 'Custom UI']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    # Not much important
    columns_to_remove = ['Bluetooth', 'Settings','Wi-Fi','Wi-Fi Features']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]
    
    return data

In [8]:
train = data_clean(train)
test = data_clean(test)

In [9]:
train = train[(train.isnull().sum(axis=1) <= 15)]


In [10]:
#the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

#the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 47
Number of data points in test: 119
Number of features in test: 46


In [11]:
def for_integer(test):
    try:
        test = test.strip()
        return int(test.split(' ')[0])
    except IOError:
           pass
    except ValueError:
        pass
    except:
        pass

def for_string(test):
    try:
        test = test.strip()
        return (test.split(' ')[0])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

def for_float(test):
    try:
        test = test.strip()
        return float(test.split(' ')[0])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass
def find_freq(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[2][0] == '(':
            return float(test[2][1:])
        return float(test[2])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

    
def for_Internal_Memory(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[1] == 'GB':
            return int(test[0])
        if test[1] == 'MB':
#             print("here")
            return (int(test[0]) * 0.001)
    except IOError:
           pass
    except ValueError:
        pass
    except:
        pass
    
def find_freq(test):
    try:
        test = test.strip()
        test = test.split(' ')
        if test[2][0] == '(':
            return float(test[2][1:])
        return float(test[2])
    except IOError:
        pass
    except ValueError:
        pass
    except:
        pass

In [12]:
def data_clean_2(x):
    data = x.copy()
    
    data['Capacity'] = data['Capacity'].apply(for_integer)

    data['Height'] = data['Height'].apply(for_float)
    data['Height'] = data['Height'].fillna(data['Height'].mean())

    data['Internal Memory'] = data['Internal Memory'].apply(for_Internal_Memory)

    data['Pixel Density'] = data['Pixel Density'].apply(for_integer)

    data['Internal Memory'] = data['Internal Memory'].fillna(data['Internal Memory'].median())
    data['Internal Memory'] = data['Internal Memory'].astype(int)

    data['RAM'] = data['RAM'].apply(for_integer)
    data['RAM'] = data['RAM'].fillna(data['RAM'].median())
    data['RAM'] = data['RAM'].astype(int)

    data['Resolution'] = data['Resolution'].apply(for_integer)
    data['Resolution'] = data['Resolution'].fillna(data['Resolution'].median())
    data['Resolution'] = data['Resolution'].astype(int)

    data['Screen Size'] = data['Screen Size'].apply(for_float)

    data['Thickness'] = data['Thickness'].apply(for_float)
    data['Thickness'] = data['Thickness'].fillna(data['Thickness'].mean())
    data['Thickness'] = data['Thickness'].round(2)

    data['Type'] = data['Type'].fillna('Li-Polymer')

    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].apply(for_float)
    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].fillna(data['Screen to Body Ratio (calculated)'].mean())
    data['Screen to Body Ratio (calculated)'] = data['Screen to Body Ratio (calculated)'].round(2)

    data['Width'] = data['Width'].apply(for_float)
    data['Width'] = data['Width'].fillna(data['Width'].mean())
    data['Width'] = data['Width'].round(2)

    data['Flash'][data['Flash'].isna() == True] = "Other"

    data['User Replaceable'][data['User Replaceable'].isna() == True] = "Other"

    data['Num_cores'] = data['Processor'].apply(for_string)
    data['Num_cores'][data['Num_cores'].isna() == True] = "Other"


    data['Processor_frequency'] = data['Processor'].apply(find_freq)
    #because there is one entry with 208MHz values, to convert it to GHz
    data['Processor_frequency'][data['Processor_frequency'] > 200] = 0.208
    data['Processor_frequency'] = data['Processor_frequency'].fillna(data['Processor_frequency'].mean())
    data['Processor_frequency'] = data['Processor_frequency'].round(2)

    data['Camera Features'][data['Camera Features'].isna() == True] = "Other"

    #simplifyig Operating System to os_name for simplicity
    data['os_name'] = data['Operating System'].apply(for_string)
    data['os_name'][data['os_name'].isna() == True] = "Other"

    data['Sim1'] = data['SIM 1'].apply(for_string)

    data['SIM Size'][data['SIM Size'].isna() == True] = "Other"

    data['Image Resolution'][data['Image Resolution'].isna() == True] = "Other"

    data['Fingerprint Sensor'][data['Fingerprint Sensor'].isna() == True] = "Other"

    data['Expandable Memory'][data['Expandable Memory'].isna() == True] = "No"

    data['Weight'] = data['Weight'].apply(for_integer)
    data['Weight'] = data['Weight'].fillna(data['Weight'].mean())
    data['Weight'] = data['Weight'].astype(int)

    data['SIM 2'] = data['SIM 2'].apply(for_string)
    data['SIM 2'][data['SIM 2'].isna() == True] = "Other"
    
    return data

In [13]:
train = data_clean_2(train)
test = data_clean_2(test)

# the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 51
Number of data points in test: 119
Number of features in test: 50


In [14]:
def data_clean_3(x):
    
    data = x.copy()

    columns_to_remove = ['User Available Storage','SIM Size','Chipset','Processor','Autofocus','Aspect Ratio','Touch Screen',
                        'Bezel-less display','Operating System','SIM 1','USB Connectivity','Other Sensors','Graphics','FM Radio',
                        'NFC','Shooting Modes','Browser','Display Colour' ]

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]


    columns_to_remove = [ 'Screen Resolution','User Replaceable','Camera Features',
                        'Thickness', 'Display Type']

    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]


    columns_to_remove = ['Fingerprint Sensor', 'Flash', 'Rating Count', 'Review Count','Image Resolution','Type','Expandable Memory',\
                        'Colours','Width','Model']
    columns_to_retain = list(set(data.columns)-set(columns_to_remove))
    data = data[columns_to_retain]

    return data

In [15]:
train = data_clean_3(train)
test = data_clean_3(test)

# the number of features and data points in train
print("Number of data points in train: %d" % train.shape[0])
print("Number of features in train: %d" % train.shape[1])

# the number of features and data points in test
print("Number of data points in test: %d" % test.shape[0])
print("Number of features in test: %d" % test.shape[1])

Number of data points in train: 341
Number of features in train: 18
Number of data points in test: 119
Number of features in test: 17


In [16]:

train_ids = train['PhoneId']
test_ids = test['PhoneId']

cols = list(test.columns)
cols.remove('PhoneId')
cols.insert(0, 'PhoneId')

combined = pd.concat([train.drop('Rating', axis=1)[cols], test[cols]])
print(combined.shape)
print(combined.columns)

combined = pd.get_dummies(combined)
print(combined.shape)
print(combined.columns)

train_new = combined[combined['PhoneId'].isin(train_ids)]
test_new = combined[combined['PhoneId'].isin(test_ids)]

(460, 17)
Index(['PhoneId', 'SIM 2', 'Sim1', 'Height', 'Brand', 'Resolution',
       'Screen Size', 'Capacity', 'Screen to Body Ratio (calculated)',
       'Pixel Density', 'os_name', 'Num_cores', 'Weight', 'Internal Memory',
       'RAM', 'Processor_frequency', 'SIM Slot(s)'],
      dtype='object')
(460, 87)
Index(['PhoneId', 'Height', 'Resolution', 'Screen Size', 'Capacity',
       'Screen to Body Ratio (calculated)', 'Pixel Density', 'Weight',
       'Internal Memory', 'RAM', 'Processor_frequency', 'SIM 2_2G', 'SIM 2_3G',
       'SIM 2_4G', 'SIM 2_Other', 'Sim1_2G', 'Sim1_3G', 'Sim1_4G',
       'Brand_10.or', 'Brand_Apple', 'Brand_Asus', 'Brand_Billion',
       'Brand_Blackberry', 'Brand_Comio', 'Brand_Coolpad', 'Brand_Do',
       'Brand_Gionee', 'Brand_Google', 'Brand_HTC', 'Brand_Honor',
       'Brand_Huawei', 'Brand_InFocus', 'Brand_Infinix', 'Brand_Intex',
       'Brand_Itel', 'Brand_Jivi', 'Brand_Karbonn', 'Brand_LG', 'Brand_Lava',
       'Brand_LeEco', 'Brand_Lenovo', 'Brand_L

In [17]:
train_new = train_new.merge(train[['PhoneId', 'Rating']], on='PhoneId')

In [18]:
# the number of features and data points in train
print("Number of data points in train: %d" % train_new.shape[0])
print("Number of features in train: %d" % train_new.shape[1])

# the number of features and data points in test
print("Number of data points in test: %d" % test_new.shape[0])
print("Number of features in test: %d" % test_new.shape[1])

Number of data points in train: 341
Number of features in train: 88
Number of data points in test: 119
Number of features in test: 87


In [19]:
train_new.head()

,PhoneId,Height,Resolution,Screen Size,Capacity,Screen to Body Ratio (calculated),Pixel Density,Weight,Internal Memory,RAM,...,Num_cores_Hexa,Num_cores_Octa,Num_cores_Other,Num_cores_Quad,Num_cores_Tru-Octa,"SIM Slot(s)_Dual SIM, GSM+CDMA","SIM Slot(s)_Dual SIM, GSM+GSM","SIM Slot(s)_Dual SIM, GSM+GSM, Dual VoLTE","SIM Slot(s)_Single SIM, GSM",Rating
0,0,157.9,20,6.26,4000,80.68,403,182,64,4,...,0,1,0,0,0,0,0,1,0,4.5
1,1,156.2,8,6.20,4230,80.85,271,168,32,3,...,0,1,0,0,0,0,0,1,0,4.5
2,2,157.0,25,6.30,3500,83.68,409,168,32,3,...,0,1,0,0,0,0,0,1,0,4.4
3,4,159.8,24,6.00,3300,74.78,411,169,64,4,...,0,1,0,0,0,0,1,0,0,4.3
4,5,160.4,16,6.50,3750,84.23,396,175,64,4,...,0,1,0,0,0,0,0,1,0,4.4


In [20]:
submission = pd.DataFrame({'PhoneId':test_new['PhoneId'], 'Class':[0]*test_new.shape[0]})
submission = submission[['PhoneId', 'Class']]
submission.head()

,PhoneId,Class
0,3,0
1,11,0
2,13,0
3,16,0
4,19,0


In [21]:
submission.to_csv("submission.csv", index=False)

In [22]:
train_binarised = train_new.apply(pd.cut, bins=2, labels=[1,0])
test_binarised = test_new.apply(pd.cut, bins=2, labels=[1,0])

In [23]:
x_train = train_new.drop('Rating', axis=1)
x_train = x_train.apply(pd.cut, bins=2, labels=[0,1])
y_train = train_new['Rating'].map(lambda x:0 if x < 4 else 1)

In [24]:
y_train[(y_train == 0)].count()

103

In [25]:
x_train = x_train.values
y_train = y_train.values

In [26]:
b = THRESHOLD
y_pred_train = []
accurate_rows = 0
for i in range(0,np.size(x_train,0)):
    if (np.sum(x_train[i,:]) >= b):
        print("MPNeuron inference is Like")
    else:
        print("MPNeuron inference is Unlike")

    if (y_train[i] == 1):
        print("Ground truth is Like")
    else:
        print("Ground truth is Unlike")

MPNeuron inference is Like
Ground truth is Like
MPNeuron inference is Like
Ground truth is Like
MPNeuron inference is Like
Ground truth is Like
MPNeuron inference is Like
Ground truth is Like
MPNeuron inference is Like
Ground truth is Like
MPNeuron inference is Like
Ground truth is Like
MPNeuron inference is Like
Ground truth is Like
MPNeuron inference is Like
Ground truth is Like
MPNeuron inference is Like
Ground truth is Like
MPNeuron inference is Like
Ground truth is Like
MPNeuron inference is Like
Ground truth is Like
MPNeuron inference is Like
Ground truth is Like
MPNeuron inference is Like
Ground truth is Like
MPNeuron inference is Like
Ground truth is Like
MPNeuron inference is Like
Ground truth is Like
MPNeuron inference is Like
Ground truth is Like
MPNeuron inference is Like
Ground truth is Like
MPNeuron inference is Like
Ground truth is Like
MPNeuron inference is Like
Ground truth is Like
MPNeuron inference is Like
Ground truth is Like
MPNeuron inference is Like
Ground truth 

In [27]:
for b in range(x_train.shape[1]):
    y_pred_train = []
    accurate_rows = 0
    for x,y in zip(x_train,y_train):
        y_pred = (np.sum(x) >= b)
        y_pred_train.append(y_pred)
        accurate_rows +=(y == y_pred)
    accuracy = accuracy_score(y_pred_train,y_train)
    print(b,accuracy)

0 0.6979472140762464
1 0.6979472140762464
2 0.6979472140762464
3 0.6979472140762464
4 0.6979472140762464
5 0.6979472140762464
6 0.6979472140762464
7 0.6891495601173021
8 0.6862170087976539
9 0.6920821114369502
10 0.6187683284457478
11 0.5601173020527859
12 0.4604105571847507
13 0.3255131964809384
14 0.3020527859237537
15 0.3020527859237537
16 0.3020527859237537
17 0.3020527859237537
18 0.3020527859237537
19 0.3020527859237537
20 0.3020527859237537
21 0.3020527859237537
22 0.3020527859237537
23 0.3020527859237537
24 0.3020527859237537
25 0.3020527859237537
26 0.3020527859237537
27 0.3020527859237537
28 0.3020527859237537
29 0.3020527859237537
30 0.3020527859237537
31 0.3020527859237537
32 0.3020527859237537
33 0.3020527859237537
34 0.3020527859237537
35 0.3020527859237537
36 0.3020527859237537
37 0.3020527859237537
38 0.3020527859237537
39 0.3020527859237537
40 0.3020527859237537
41 0.3020527859237537
42 0.3020527859237537
43 0.3020527859237537
44 0.3020527859237537
45 0.302052785923753

In [28]:
class MPNeuron():
    def __init__(self):
        self.b = None
    def model(self, x):
        return (np.sum(x) >= self.b)
    def predict(self, X):
        y = []
        for x in X:
            result = self.model(x)
            y.append(result)
        return np.array(y)
    def fit(self, X, y):
        accuracy = {}
        for b in range(X.shape[1]+1):
            self.b = b
            y_pred = self.predict(X)
            accuracy[b] = accuracy_score(y_pred, y)
        best_b = max(accuracy, key=accuracy.get)
        self.b = best_b
        print("optimal value of b is ", best_b)
        print("highest accuracy is ", accuracy[best_b])
mp_neuron = MPNeuron()
mp_neuron.fit(x_train, y_train)

optimal value of b is  0
highest accuracy is  0.6979472140762464
